In [1]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# CHSHゲームの量子回路を生成する関数
def make_chsh_circuit(x, y):
    """
    入力 x, y に基づいてCHSHゲームの量子回路を作成する
    """
    qc = QuantumCircuit(2, 2)
    
    # 1. もつれ状態（ベル状態）を生成
    qc.h(0)
    qc.cx(0, 1)
    qc.barrier()

    # 2. 入力に応じてアリスが測定軸を回転
    if x == 1:
        qc.ry(-np.pi / 2, 0) # Z基底からX基底へ回転

    # 3. 入力に応じてボブが測定軸を回転
    if y == 0:
        qc.ry(-np.pi / 4, 1)
    else:
        qc.ry(np.pi / 4, 1)
        
    qc.barrier()
    
    # 4. 両方の量子ビットを測定
    qc.measure([0, 1], [0, 1])
    
    return qc

# --- メインの実行部分 ---

# シミュレータを準備
simulator = AerSimulator()
SHOTS = 4000  # 試行回数

total_win_probability = 0

print("CHSHゲームのシミュレーションを開始します...")
print("-" * 30)

# 4つの入力パターン (x, y) をループ
for x in [0, 1]:
    for y in [0, 1]:
        # 回路を生成
        circuit = make_chsh_circuit(x, y)
        
        # 回路をシミュレータ用に変換
        t_circuit = transpile(circuit, simulator)
        
        # 実行して結果を取得
        result = simulator.run(t_circuit, shots=SHOTS).result()
        counts = result.get_counts(circuit)
        
        # このケースでの勝利数を計算
        win_condition = x * y
        win_count = 0
        
        for outcome, count in counts.items():
            # Qiskitの出力は 'b a' の順なので注意
            b = int(outcome[0])
            a = int(outcome[1])
            
            # 勝利条件 a ⊕ b == x · y をチェック
            if (a ^ b) == win_condition:
                win_count += count
        
        win_probability = win_count / SHOTS
        total_win_probability += win_probability
        
        print(f"入力 (x={x}, y={y}):")
        print(f"  勝利条件: a ⊕ b = {win_condition}")
        print(f"  勝率: {win_probability:.4f} ({win_count}/{SHOTS})")

print("-" * 30)

# 4ケースの平均勝率を計算
average_win_probability = total_win_probability / 4
print(f"📈 平均勝率: {average_win_probability:.4f}")
print(f"   (理論値: {np.cos(np.pi/8)**2:.4f})")

if average_win_probability > 0.75:
    print("\n🎉 おめでとうございます！古典限界の75%を超えることに成功しました！")
else:
    print("\n🤔 75%を超えませんでした。コードや理論を見直してみましょう。")

CHSHゲームのシミュレーションを開始します...
------------------------------
入力 (x=0, y=0):
  勝利条件: a ⊕ b = 0
  勝率: 0.8510 (3404/4000)
入力 (x=0, y=1):
  勝利条件: a ⊕ b = 0
  勝率: 0.8510 (3404/4000)
入力 (x=1, y=0):
  勝利条件: a ⊕ b = 0
  勝率: 0.8592 (3437/4000)
入力 (x=1, y=1):
  勝利条件: a ⊕ b = 1
  勝率: 0.8470 (3388/4000)
------------------------------
📈 平均勝率: 0.8521
   (理論値: 0.8536)

🎉 おめでとうございます！古典限界の75%を超えることに成功しました！


In [5]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error

# --- パラメータ設定 ---
# デコヒーレンスの強さを設定 (0.0はノイズなし, 0.1は10%のエラー確率)
noise_level = 0.1
shots = 4000

def make_chsh_circuit(x: int, y: int) -> QuantumCircuit:
    """
    入力 x, y に基づいてQiskitでCHSHゲームの量子回路を作成する
    """
    qc = QuantumCircuit(2, 2)
    
    # 1. エンタングルメント状態を生成
    qc.h(0)
    qc.cx(0, 1)
    qc.barrier()
    
    # 2. アリスの測定基底を回転
    if x == 1:
        qc.ry(-np.pi / 2, 0)
        
    # 3. ボブの測定基底を回転
    if y == 0:
        qc.ry(-np.pi / 4, 1)
    else:
        qc.ry(np.pi / 4, 1)
        
    qc.barrier()
    
    # 4. 測定
    qc.measure(0, 0) # アリスの測定結果 a
    qc.measure(1, 1) # ボブの測定結果 b
    
    return qc

def create_noise_model(error_prob: float) -> NoiseModel | None:
    """
    指定されたエラー確率でデコヒーレンスノイズモデルを作成する
    """
    if error_prob == 0:
        return None # ノイズなし
        
    # 1量子ビットおよび2量子ビットゲートに適用するデコヒーレンスエラーを定義
    error_1 = depolarizing_error(error_prob, 1)
    error_2 = depolarizing_error(error_prob, 2)
    
    noise_model = NoiseModel()
    # 全ての単一量子ビットゲートと2量子ビットゲートにエラーを追加
    noise_model.add_all_qubit_quantum_error(error_1, ['h', 'ry'])
    noise_model.add_all_qubit_quantum_error(error_2, ['cx']) # CNOTゲート
    
    return noise_model

# --- シミュレーションの実行 ---

# 1. ノイズモデルを作成
noise = create_noise_model(noise_level)

# 2. ノイズモデルを設定したシミュレータを準備
simulator = AerSimulator(noise_model=noise)

total_win_count = 0

print(f"CHSHゲームのシミュレーションを開始します...")
print(f"設定されたノイズレベル: {noise_level * 100:.1f}%")
print(f"ショット数: {shots}")
print("-" * 50)

# 4つの入力(x, y)の組み合わせでループ
for x in [0, 1]:
    for y in [0, 1]:
        # 回路を作成
        chsh_circuit = make_chsh_circuit(x, y)
        
        # シミュレーションを実行
        job = simulator.run(chsh_circuit, shots=shots)
        result = job.result()
        counts = result.get_counts()
        
        # 勝利条件 a + b (mod 2) == x * y に基づいて勝利数を計算
        # Qiskitの測定結果は 'ba' の順（q1が左）で返ってくる
        win_count_for_case = 0
        for outcome, count in counts.items():
            b, a = int(outcome[0]), int(outcome[1])
            if (a + b) % 2 == x * y:
                win_count_for_case += count
        
        win_probability_for_case = win_count_for_case / shots
        print(f"入力 (x={x}, y={y}): 勝率: {win_probability_for_case:.4f} ({win_count_for_case}/{shots})")
        
        total_win_count += win_count_for_case

print("-" * 20)
average_win_probability = total_win_count / (shots * 4)
theoretical_max_win_prob = np.cos(np.pi / 8)**2
classical_limit = 0.75

print(f"✅ 全体の平均勝率: {average_win_probability:.4f}")
print(f"💡 理論上の最大勝率 (量子): {theoretical_max_win_prob:.4f} (cos^2(π/8))")
print(f"🔒 古典戦略の限界勝率: {classical_limit:.4f}")

CHSHゲームのシミュレーションを開始します...
設定されたノイズレベル: 10.0%
ショット数: 4000
--------------------------------------------------
入力 (x=0, y=0): 勝率: 0.7835 (3134/4000)
入力 (x=0, y=1): 勝率: 0.7875 (3150/4000)
入力 (x=1, y=0): 勝率: 0.7165 (2866/4000)
入力 (x=1, y=1): 勝率: 0.7260 (2904/4000)
--------------------
✅ 全体の平均勝率: 0.7534
💡 理論上の最大勝率 (量子): 0.8536 (cos^2(π/8))
🔒 古典戦略の限界勝率: 0.7500
